In [1]:
import time
import sys
from projectq import MainEngine
from projectq.ops import H, Measure, All, CNOT
from projectq.backends import Simulator
from projectq.cengines import TagRemover

# Helper function for Quantum Fourier Transform
def apply_qft(eng, qubits):
    n = len(qubits)
    for i in range(n):
        H | qubits[i]
        for j in range(i + 1, n):
            CNOT | (qubits[j], qubits[i])

# Helper function for Inverse Quantum Fourier Transform (QFT†)
def apply_inverse_qft(eng, qubits):
    n = len(qubits)
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            CNOT | (qubits[j], qubits[i])
        H | qubits[i]

# Double AES with QFT for two 16-bit keys
def Double_AES_QFT(eng, resource_check):
    # Step 1: Allocate registers for 16-bit plaintext and two 16-bit keys
    x = eng.allocate_qureg(16)   # 16-bit plaintext
    k1 = eng.allocate_qureg(16)  # First 16-bit key
    k2 = eng.allocate_qureg(16)  # Second 16-bit key

    # Step 2: Apply QFT to plaintext and keys
    print("Applying QFT to plaintext and keys...")
    apply_qft(eng, x)
    apply_qft(eng, k1)
    apply_qft(eng, k2)
    eng.flush()

    # Step 3: First AES-16 Layer
    print("Running First AES Layer (AES-16)")
    AES_16(eng, resource_check, x, k1)

    # Step 4: Apply inverse QFT to plaintext and keys
    print("Applying Inverse QFT to ciphertext and keys...")
    apply_inverse_qft(eng, x)
    apply_inverse_qft(eng, k1)
    apply_inverse_qft(eng, k2)
    eng.flush()

    # Step 5: Second AES-16 Layer with the second key
    print("Running Second AES Layer (AES-16)")
    AES_16(eng, resource_check, x, k2)

    # Step 6: Final inverse QFT on the ciphertext
    print("Applying final Inverse QFT to output...")
    apply_inverse_qft(eng, x)
    eng.flush()

    # Output final state
    print("Final Ciphertext:")
    print_state(eng, x, 4)

    return x

# AES-16 function with a 16-bit key
def AES_16(eng, resource_check, x, k):
    # Initial XOR with round constants for state and key
    if resource_check != 1:
        Round_constant_XOR(eng, x, 0x1234, 16)
        Round_constant_XOR(eng, k, 0x1234, 16)

    # Encryption rounds (2 rounds for AES-16)
    for i in range(2):
        Keyshedule_16(eng, k, i, resource_check)
        SBox_bp12_all(eng, x, resource_check)
        x = Maxi_mc(eng, x) if i != 1 else x
        AddRoundkey(eng, x, k)

    return x

# Helper functions
def Keyshedule_16(eng, k, round, resource_check):
    pass

def SBox_bp12_all(eng, x, resource_check):
    pass

def Maxi_mc(eng, x):
    return x

def AddRoundkey(eng, x, k):
    CNOT16(eng, k, x)

def Round_constant_XOR(eng, x, constant, bits):
    for i in range(bits):
        if (constant >> i) & 1:
            X | x[i]

def CNOT16(eng, a, b):
    for i in range(16):
        CNOT | (a[i], b[i])

def print_state(eng, qubits, n):
    eng.flush()
    state = []
    for i in range(n):
        state.append(int(qubits[i]))

    binary_str = ''.join(str(bit) for bit in state)
    hex_str = hex(int(binary_str, 2))[2:].zfill((n + 3) // 4)
    print(f'Hex: {hex_str.upper()}')
    print(f'Binary: {binary_str}')
    sys.stdout.flush()

# Main function to run Double AES with QFT
def main():
    start_time = time.time()

    sim = Simulator()
    eng = MainEngine(backend=sim, engine_list=[TagRemover()])

    Double_AES_QFT(eng, 0)

    end_time = time.time()
    execution_time = end_time - start_time
    print(f'\nExecution Time: {execution_time:.4f} seconds')
    sys.stdout.flush()

if __name__ == '__main__':
    main()


(Note: This is the (slow) Python simulator.)


MemoryError: cannot allocate memory for array
 raised in:
'  File "C:\\Users\\jimmy\\Documents\\KULIAH S2\\THESIS\\Penelitian Bab 4\\myenv\\Lib\\site-packages\\projectq\\backends\\_sim\\_pysim.py", line 114, in allocate_qubit'
'    self._state.resize(1 << self._num_qubits, refcheck=_USE_REFCHECK)'